# Loading a CITE Library



## Configuring CITE libraries for almond kernel

First, we'll make a bintray repository with CITE libraries available to your almond kernel.

In [1]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [2]:
import $ivy.`edu.holycross.shot::ohco2:10.16.0`
import $ivy.`edu.holycross.shot.cite::xcite:4.1.1`
import $ivy.`edu.holycross.shot::scm:7.2.0`
import $ivy.`edu.holycross.shot::dse:5.2.2`
import $ivy.`edu.holycross.shot::citebinaryimage:3.1.1`
import $ivy.`edu.holycross.shot::citeobj:7.3.4`
import $ivy.`edu.holycross.shot::citerelations:2.5.2`
import $ivy.`edu.holycross.shot::cex:6.3.3`


import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                              

import $ivy.$                              

import $ivy.$                                          

import $ivy.$                                  

import $ivy.$                                        

import $ivy.$                              


## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [3]:
// Import some CITE libraries
import edu.holycross.shot.cite._
import edu.holycross.shot.ohco2._
import edu.holycross.shot.scm._
import edu.holycross.shot.citeobj._
import edu.holycross.shot.citerelation._
import edu.holycross.shot.dse._
import edu.holycross.shot.citebinaryimage._
import edu.holycross.shot.ohco2._

import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source


import edu.holycross.shot.cite._

import edu.holycross.shot.ohco2._

import edu.holycross.shot.scm._

import edu.holycross.shot.citeobj._

import edu.holycross.shot.citerelation._

import edu.holycross.shot.dse._

import edu.holycross.shot.citebinaryimage._

import edu.holycross.shot.ohco2._


import almond.display.UpdatableDisplay

import almond.interpreter.api.DisplayData.ContentType

import almond.interpreter.api.{DisplayData, OutputHandler}


import java.io.File

import java.io.PrintWriter


import scala.io.Source


## Useful Functions

Save a string to a names file:

In [4]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

defined function saveString

Convert a Roman Numeral to an Integer:

In [5]:
def fromRoman(s: String) : Int = {
	try {
		val numerals = Map('I' -> 1, 'V' -> 5, 'X' -> 10, 'L' -> 50, 'C' -> 100, 'D' -> 500, 'M' -> 1000)

		s.toUpperCase.map(numerals).foldLeft((0,0)) {
		  case ((sum, last), curr) =>  (sum + curr + (if (last < curr) -2*last else 0), curr) }._1
	} catch {
		case e:Exception => throw new Exception(s""" "${s}" is not a valid Roman Numeral.""")
	}
}

defined function fromRoman

## Assemble a Library

Build a CITE Library by aggregating files, to incorporate ongoing editing.

## Load the New Library

Load a CITE Library…

In [23]:
val cexPath = "https://raw.githubusercontent.com/Furman-Editions-In-Progress/xolotl/master/cex/xolotl_all.cex"
val lib = CiteLibrary(scala.io.Source.fromURL(cexPath).mkString)

Feb 04, 2020 10:25:36 PM wvlet.log.Logger log
INFO: Building text repo from cex ...
Feb 04, 2020 10:25:36 PM wvlet.log.Logger log
INFO: Building collection repo from cex ...
Feb 04, 2020 10:25:36 PM wvlet.log.Logger log
INFO: Building relations from cex ...
Feb 04, 2020 10:25:36 PM wvlet.log.Logger log
INFO: All library components built.


cexPath: String = "https://raw.githubusercontent.com/Furman-Editions-In-Progress/xolotl/master/cex/xolotl_all.cex"
lib: CiteLibrary = CiteLibrary(
  "Codex Xolotl, Initial data release, 2020d (Contains dummy data for testing!)",
  Cite2Urn("urn:cite2:cex:temp_xolotl.v1:2020d"),
  "CC 3.0 NC-BY",
  Vector(),
  Some(
    TextRepository(
      Corpus(
        Vector(
          CitableNode(
            CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.0"),
            "De "
          ),
          CitableNode(
            CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.1"),
            "c\u00f3mo "
          ),
          CitableNode(
            CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.2"),
            "el "
          ),
          CitableNode(
            CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.3"),
            "chichimeca "
          ),
          CitableNode(
            CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.4"),

### Get the Components of the Library

**Warning** The code below makes some assumptions about this particular CiteLibrary, namely that it has a TextRepository and a CollectionRepository. Those `.get` methods will throw an exception in the even that the loaded library has `None` as the value for `.textRepository` or `.collectionRepository`.

In [16]:
val tr = lib.textRepository.get
val cat = tr.catalog
val corp = tr.corpus
val cr = lib.collectionRepository.get

tr: TextRepository = TextRepository(
  Corpus(
    Vector(
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.0"),
        "De "
      ),
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.1"),
        "c\u00f3mo "
      ),
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.2"),
        "el "
      ),
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.3"),
        "chichimeca "
      ),
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.4"),
        "Xolotl, "
      ),
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.5"),
        "se\u00f1or "
      ),
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.6"),
        "de "
      ),
      CitableNode(
        CtsUrn("urn:cts:xolotl:torquemada.001.offner.token:15.head.7"),
        "las "
      ),


## Find Things in a Scene

Get the URN for a scene we are interested in.

In [17]:
val sceneComment = Cite2Urn("urn:cite2:xolotl:commentary.v1:1")
val sceneImgProperty = cr.data ~~ Cite2Urn("urn:cite2:xolotl:commentary.v1.img:1")
val sceneImg = sceneImgProperty.data.head.propertyValue
val sceneImgROI = sceneImg.asInstanceOf[Cite2Urn].objectExtensionOption
val sceneRoi = ImageROI(sceneImgROI.get)

sceneComment: Cite2Urn = Cite2Urn("urn:cite2:xolotl:commentary.v1:1")
sceneImgProperty: CiteCollectionData = CiteCollectionData(
  Vector(
    CitePropertyValue(
      Cite2Urn("urn:cite2:xolotl:commentary.v1.img:1"),
      Cite2Urn(
        "urn:cite2:xolotl:codeximg.2020:27_11r_vis2@0.3374,0.1747,0.2732,0.1129"
      )
    )
  )
)
sceneImg: Any = Cite2Urn(
  "urn:cite2:xolotl:codeximg.2020:27_11r_vis2@0.3374,0.1747,0.2732,0.1129"
)
sceneImgROI: Option[String] = Some("0.3374,0.1747,0.2732,0.1129")
sceneRoi: ImageROI = ImageROI("0.3374,0.1747,0.2732,0.1129")

## Data Models

In [18]:
val extendedTextDM = Cite2Urn("urn:cite2:cite:datamodels.v1:extensions_text")

extendedTextDM: Cite2Urn = Cite2Urn(
  "urn:cite2:cite:datamodels.v1:extensions_text"
)

In [19]:


val extTextColls: Vector[Cite2Urn] = {
    lib.dataModels.getOrElse(Vector[DataModel]()).filter(_.model == extendedTextDM).map(_.collection)
}

extTextColls: Vector[Cite2Urn] = Vector(
  Cite2Urn("urn:cite2:fufolio:extended_text_properties.v1:")
)

In [20]:
val extObjs = cr ~~ Cite2Urn("urn:cite2:fufolio:extended_text_properties.v1:")

extObjs: Vector[CiteObject] = Vector(
  CiteObject(
    Cite2Urn("urn:cite2:fufolio:extended_text_properties.v1:xolotl_places"),
    "Place identifications proposed for the CodexXolotl",
    Vector(
      CitePropertyImplementation(
        Cite2Urn(
          "urn:cite2:fufolio:extended_text_properties.v1.extendedtype:xolotl_places"
        ),
        CitePropertyDef(
          Cite2Urn(
            "urn:cite2:fufolio:extended_text_properties.v1.extendedtype:"
          ),
          "Extended Text Type",
          ControlledVocabType,
          Vector("geojson", "pleiadesuri", "latlong", "markdown", "teixml")
        ),
        "latlong"
      ),
      CitePropertyImplementation(
        Cite2Urn(
          "urn:cite2:fufolio:extended_text_properties.v1.propertyurn:xolotl_places"
        ),
        CitePropertyDef(
          Cite2Urn("urn:cite2:fufolio:extended_text_properties.v1.propertyurn:"),
          "Property URN",
          Cite2UrnType,
          Vector()
        ),
        Ci

In [21]:
extObjs.map( o => {
    val pu = o.urn.addProperty("propertyurn")
    val su = o.urn.addProperty("extendedtype")
    (pu, o.propertyValue(su))
})

res20: Vector[(Cite2Urn, Any)] = Vector(
  (
    Cite2Urn(
      "urn:cite2:fufolio:extended_text_properties.v1.propertyurn:xolotl_places"
    ),
    "latlong"
  ),
  (
    Cite2Urn(
      "urn:cite2:fufolio:extended_text_properties.v1.propertyurn:xolotl_md"
    ),
    "markdown"
  )
)